In [1]:
import pandas as pd
df = pd.read_csv("Online Retail.csv", encoding='latin1', parse_dates=['InvoiceDate'])
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [2]:
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,01-12-2010 08:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,01-12-2010 08:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,01-12-2010 08:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,01-12-2010 08:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,01-12-2010 08:34,1.69,13047.0,United Kingdom


In [3]:
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [4]:
df = df[(df['Quantity']>0) & (df['UnitPrice']>0)]

In [5]:
df = df.dropna(subset=['CustomerID'])

In [6]:
import warnings
warnings.filterwarnings('ignore')
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [7]:
df['CustomerID'] = df['CustomerID'].astype(int)


In [8]:
top_customers = df.groupby('CustomerID')['TotalPrice'].sum().sort_values(ascending=False).head(10)
top_customers

CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
12415    124914.53
14156    117379.63
17511     91062.38
16029     81024.84
12346     77183.60
Name: TotalPrice, dtype: float64

In [9]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')


In [10]:
monthly = df.resample('M', on='InvoiceDate')['TotalPrice'].sum()
monthly

InvoiceDate
2010-01-31     46376.49
2010-02-28     47316.53
2010-03-31     23921.71
2010-04-30         0.00
2010-05-31     31771.60
2010-06-30     31215.64
2010-07-31     53795.31
2010-08-31     39248.82
2010-09-30     38231.90
2010-10-31     33650.28
2010-11-30         0.00
2010-12-31     17305.77
2011-01-31    239994.38
2011-02-28    209542.22
2011-03-31    261508.44
2011-04-30    329347.22
2011-05-31    322262.21
2011-06-30    305544.20
2011-07-31    322993.67
2011-08-31    257498.63
2011-09-30    421984.18
2011-10-31    291455.90
2011-11-30    345332.09
2011-12-31    205190.80
Name: TotalPrice, dtype: float64

In [11]:
top_products = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)

In [12]:
top_products

Description
PAPER CRAFT , LITTLE BIRDIE           168469.60
REGENCY CAKESTAND 3 TIER              142592.95
WHITE HANGING HEART T-LIGHT HOLDER    100448.15
JUMBO BAG RED RETROSPOT                85220.78
MEDIUM CERAMIC TOP STORAGE JAR         81416.73
POSTAGE                                77803.96
PARTY BUNTING                          68844.33
ASSORTED COLOUR BIRD ORNAMENT          56580.34
Manual                                 53779.93
RABBIT NIGHT LIGHT                     51346.20
Name: TotalPrice, dtype: float64

In [17]:
rfm = rfm.dropna(subset=['Recency'])



In [18]:
rfm['R_rank'] = pd.qcut(rfm['Recency'], 4, labels=[4,3,2,1], duplicates='drop')
rfm['F_rank'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1,2,3,4], duplicates='drop')
rfm['M_rank'] = pd.qcut(rfm['Monetary'], 4, labels=[1,2,3,4], duplicates='drop')

rfm[['R_rank','F_rank','M_rank']] = rfm[['R_rank','F_rank','M_rank']].astype(int)

rfm['RFM_Score'] = (
    rfm['R_rank'].astype(str) +
    rfm['F_rank'].astype(str) +
    rfm['M_rank'].astype(str)
)


In [19]:
print(rfm.head())
print(rfm['RFM_Score'].value_counts())


   CustomerID  Recency  Frequency  Monetary  R_rank  F_rank  M_rank RFM_Score
1       12347     96.0          7   4310.00       3       4       4       344
2       12348    221.0          4   1797.24       1       3       3       133
4       12350    312.0          1    334.40       1       1       1       111
5       12352    275.0          8   2506.04       1       4       4       144
8       12355     97.0          1    459.40       3       1       2       312
RFM_Score
444    263
111    216
344    162
211    143
433    115
      ... 
314      2
414      2
431      1
141      1
241      1
Name: count, Length: 62, dtype: int64


In [21]:
rfm.to_csv('RFM cleaned_data.csv', index=False)
